In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:85% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font color="red" size="6">ch14. 웹데이터 수집</font></b>
# 1절. BeautifulSoup과 parser
```pip install bs4``` 아나콘다를 설치하면 7500개여개의 패키지 설치
- 공식 사이트 : https://www.crummy.com/software/BeautifulSoup/
- Docs : https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [2]:
import requests # HTTP 요청 처리 lib
from requests_file import FileAdapter

In [3]:
s = requests.Session() # HTTP 요청관리를 위한 세션 객체
s.mount("file://", FileAdapter())
response = s.get('file:///ai_x/lecNote/01_python/data/ch14_sample.html')
response

<Response [404]>

In [4]:
response.status_code 
# 200 : 정상
# 404 : 없는 페이지
# 406 : get, post 오류

404

In [5]:
response.content # 바이너리 형식의 내용

b"[Errno 2] No such file or directory: '\\\\ai_x\\\\lecNote\\\\01_python\\\\data\\\\ch14_sample.html'"

In [6]:
response.content.decode('utf-8')

"[Errno 2] No such file or directory: '\\\\ai_x\\\\lecNote\\\\01_python\\\\data\\\\ch14_sample.html'"

In [7]:
response.text

"[Errno 2] No such file or directory: '\\\\ai_x\\\\lecNote\\\\01_python\\\\data\\\\ch14_sample.html'"

In [8]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content,  # response.text
                    "html.parser")
# soup

C:\Users\Admin\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [9]:
# soup.select_one('선택자') : 해당 선택자 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1태그 하나만
print('el :', el)
print('el.text :', el.text)
print('el.string :', el.string)
print('el의 속성들 :', el.attrs)
print('el의 title속성 :', el.attrs['title'])
print('el의 title속성 :', el.attrs.get('title') )
print('el의 이름 :', el.name)

el : None


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
# soup.select('선택자') : 해당 선택자 모든 엘리먼트를 리스트로
el = soup.select('h1')
print('리스트 el :', el)
print('el의 text들 :', [e.text for e in el])
# for e in el:
#     print(e.text, end=',')
print('el의 속성들 :', [e.attrs for e in el])
print('el의 class속성들 :', [e.attrs.get('class') for e in el])

In [ ]:
# select_one(선택자)와 find(태그, 속성)
print('select_one :', soup.select_one('h1.css'))
print('find :', soup.find('h1', {'class':'css'}))
print('find :', soup.find('h1', class_='css'))
print()
print('select_one :', soup.select_one('h1#text'))
print('find :', soup.find('h1', {'id':'text'}));

In [ ]:
# select(선택자)와 find_all(태그, 속성)
print('모든 h1.css, span 태그(select) :',
     soup.select('h1.css, span'))
print('모든 h1.css, span태그(find_all) :',
     soup.find_all(['h1','span'], class_='css'))

In [ ]:
# 없는 엘리먼트 찾기
print('find_all(빈list) :', soup.find_all('a', class_='css'))
print('find(None) :', soup.find('a', class_='css'))
print('select(빈list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))

# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
- json, html, xml
## 2.1 JSON파일
- request모듈(get)
- urllib.request모듈(urlopen)

In [ ]:
# 크롤링 허용 범위는 사이트마다 ~/robots.txt에서 확인할 수 있습니다

In [ ]:
# 방법1
import requests
response = requests.get('http://api.github.com')
response, response.status_code

In [ ]:
# 방법2
from urllib.request import urlopen
response = urlopen('http://api.github.com')
response

In [ ]:
text = '{"속성1": "값1","속성2":"값2"}'
type(text)

In [ ]:
# 문자(딕셔너리 타입)를 딕셔너리
# '{"속성1": "값1","속성2":"값2"}' => {"속성1": "값1","속성2":"값2"}
import json
json.loads(text)

## 2.2 html 파일
### 1) 환율정보 가져오기 (네이버->증권->시장지표)
- https://finance.naver.com/marketindex/

In [ ]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex'
response = requests.get(url)
# response, response.status_code
# response.content 
# response.content.decode('cp949') == response.text
soup = BeautifulSoup(response.text, # response.content
                    'html.parser')

In [ ]:
# 방법2
from urllib.request import urlopen
response = urlopen(url)
# response.status 상태코드
# response.read()
# response.read().decode('cp949')
soup = BeautifulSoup(response,  # response.read()
                     'html.parser')

In [ ]:
title = soup.select('h3.h_lst > span.blind')
for t in title:
    print(t.text, end='\t')

In [ ]:
price = soup.select('div.head_info > span.value')
[p.text for p in price]
[round(float(p.text.replace(',',''))) for p in price]
[round(float(''.join(p.text.split(',')))) for p in price]

In [ ]:
out = '1,374.70'
round(float(''.join(out.split(','))))

In [ ]:
round(float(out.replace(',','')))

In [ ]:
unit = soup.select('div.head_info > span > span.blind')
unit = [u.text for u in unit]
unit.insert(7, '') # 7번째 index에 '' 추가
unit

In [ ]:
status = soup.select('div.head_info > span.blind')
[t.text for t in title]
[p.text for p in price]
unit
[s.string for s in status]

In [ ]:
len(title), len(price), len(unit), len(status)

In [ ]:
for idx in range(len(title)):
    print("{}. {} : {}{} - {}".format(idx+1,
                                     title[idx].text,
                                     price[idx].text,
                                     unit[idx],
                                     status[idx].text))

In [ ]:
for t, p, u, s  in zip(title, price, unit, status):
    print("{} : {}{} - {}".format(t.text,
                                 p.text,
                                 u,
                                 s.text))

In [ ]:
for idx, (t, p, u, s)  in enumerate(zip(title, price, unit, status)):
    print("{}. {} : {}{} - {}".format(idx+1,
                                t.text,
                                p.text,
                                u,
                                s.text))

### 2) 이번주 로또번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin(google에 "로또번호 당첨번호" 검색)
```
    1173회(2025년 05월 24일 추첨)
    당첨번호 [1 5 18 20 30 35]
    보너스 3
```

In [ ]:
# 방법1의 soup객체 생성
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.content, # response.text
                    "html.parser")

In [ ]:
# 방법2의 sou객체 생성
from urllib.request import urlopen
response = urlopen(url)
soup = BeautifulSoup(response, "html.parser")

In [ ]:
times = soup.select_one('div.win_result > h4 > strong').text # 1173회
date = soup.select_one('div.win_result >p.desc').text  #(2025년 05월 24일 추첨)
print(times, date)
title1 = soup.select_one('div.num.win > strong').text
lotto_number = soup.select('div.num.win > p > span')
print(title1, [int(lotto.text) for lotto in lotto_number])
title2 = soup.select_one('div.num.bonus > strong').string
bonus_number = soup.select_one('div.num.bonus > p > span').text
print(title2, bonus_number)

In [ ]:
# 위의 select 함수를 find함수로
#times = soup.select_one('div.win_result > h4 > strong').text # 1173회
win_result = soup.find('div', class_="win_result")
times = win_result.find('strong').text
# date = soup.select_one('div.win_result >p.desc').text  #(2025년 05월 24일 추첨)
date = soup.find('p', class_='desc').text
print(times, date)
# title1 = soup.select_one('div.num.win > strong').text
num_win = soup.find('div', class_=['win'])
title1 = num_win.find('strong').text
# lotto_number = soup.select('div.num.win > p > span')
lotto_number = num_win.find_all('span') # 배열
print(title1, [int(lotto.text) for lotto in lotto_number])
# title2 = soup.select_one('div.num.bonus > strong').string
num_bonus = soup.find('div', class_=['bonus'])
title2 = num_bonus.find('strong').text
# bonus_number = soup.select_one('div.num.bonus > p > span').text
bonus_number = num_bonus.find('span').text
print(title2, bonus_number)

### 3) 다음 검색 리스트
```
no  title   link
0 [비트코인 2025] 백악관 크립토 차르~ https://v.daum.net/v/20250528103907230
1 [비즈 나우] 비트코인 2025 컨퍼런~   https://v.daum.net/v/20250528075215864
....
```

In [ ]:
# 방법1
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q='+word
response = requests.get(url)
soup = BeautifulSoup(response.text, # response.content
                    "html.parser")
items_find_list = [] # 검색한 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
# len(items_el)
for idx, item in enumerate(items_el):
    items_find_list.append({'no': idx,
                           'title': item.text,
                           'link': item.attrs['href']})
import pandas as pd
pd.DataFrame(items_find_list)

In [ ]:
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q='+word
response = requests.get(url)
soup = BeautifulSoup(response.text, # response.content
                    "html.parser")
items_find_list = [] # 검색한 결과를 담을 2차원 리스트
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append([idx, item.text, item.attrs.get('href')])
# items_find_list
import pandas as pd
pd.DataFrame(items_find_list, columns=['no', 'title', 'link'])

In [ ]:
# 다음 뉴스 검색(키워드, 원하는 페이지수)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
keyword = '비트코인'
page = 2
# url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&q={keyword}&p={page}'
# response = requests.get(url)
url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y'
params = {'q':keyword, 'p':page}
response = requests.get(url, params=params)
soup = BeautifulSoup(response.text,  # response.content
                    'html.parser')
items_find_list = [] # 검색한 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
# len(items_el)
for idx, item in enumerate(items_el):
    items_find_list.append({'no': (page-1)*10 + idx,
                           'title': item.text,
                           'link': item.attrs['href']})
    print({'no': (page-1)*10 + idx,
           'title': item.text,
           'link': item.attrs['href']})

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
def collect_list(keyword, page=1):
    'keyword로 다음 검색한 결과(해당 page)를 return'
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text,  # response.content
                    'html.parser')
    items_find_list = [] # 검색한 결과를 담을 dict list
    items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
    for idx, item in enumerate(items_el):
        items_find_list.append({'no': (page-1)*10 + idx,
                           'title': item.text,
                           'link': item.attrs['href']})
    return items_find_list

In [ ]:
collect_list('청바지',2)

In [ ]:
# 원하는 keyword로 다음검색
result = []
keyword = "청바지"
pages = 4
for page in range(1, pages+1):
    result.extend(collect_list(keyword, page))
    time.sleep(3)
result_df = pd.DataFrame(result)
result_df.head()

In [ ]:
keywords = ['청바지','면바지']
pages = 2
result0 = [] # 청바지 검색결과 50개
result1 = [] # 면바지 검색결과 50개
for i, keyword in enumerate(keywords):
    for page in range(1, pages+1):
        print(f'~ ~ ~ {i}번째 {keyword} {page} 검색 중입니다 ~ ~ ~')
        if i==0:
            result0.extend(collect_list(keyword, page))
        else:
            result1.extend(collect_list(keyword, page))
        time.sleep(4)

In [ ]:
result0_df = pd.DataFrame(result0)
result0_df.head()

In [ ]:
result1_df = pd.DataFrame(result1)
result1_df.head()

In [ ]:
result0_df.to_csv('data/ch14_'+keywords[0]+'.csv', index=False, encoding='cp949')
result1_df.to_csv('data/ch14_'+keywords[1]+'.csv', index=False)

### 4) User-agent를 추가하여 크롤링
- urlopen()함수를 사용하면 크롤링이 안 되는 사이트
- User-agent를 추가하여 크롤링

In [ ]:
# 방법2
from urllib.request import urlopen, Request
import urllib.parse
word = '비트코인'
word = urllib.parse.quote(word)
# print(word)
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q='+word
# User-Agent를 추가하여 브라우저처럼 보이게 포장
headers = {'user-agent':
         'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
response = urlopen(request)
soup = BeautifulSoup(response, # response.content
                    "html.parser")
def collect_list(keyword, page=1):
    'keyword로 다음 검색한 결과(해당 page)를 return'
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text,  # response.content
                    'html.parser')
    items_find_list = [] # 검색한 결과를 담을 dict list
    items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
    for idx, item in enumerate(items_el):
        items_find_list.append({'no': (page-1)*10 + idx,
                           'title': item.text,
                           'link': item.attrs['href']})
    return items_find_list
collect_list("청바지", 1)

- 꼭 User-Agent를 사용하여야 하는 경우 : https://www.melon.com/chart/

In [ ]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://www.melon.com/chart/'
melonpage = requests.get(url)
melonpage.status_code

In [ ]:
# 방법2
from urllib.request import urlopen, Request
# melonpage = urlopen(url) 에러남

In [ ]:
# User-Agent 추가
headers = {'user-agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'
}
# 방법2
request = Request(url, headers=headers)
melonpage = urlopen(request)
# 방법1
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, # melonpage.content
                    "html.parser")
#soup

In [ ]:
# 멜론 순위, 노래제목, 가수, 좋아요갯수
# 1위 | 너에게 닿기를 | 10cm (100위까지 크롤링)
title_els = soup.select('div.ellipsis.rank01 > span')
singer_els = soup.select('div.ellipsis.rank02 > span.checkEllipsis')
len([title.text.strip() for title in title_els]), \
len([singer.text for singer in singer_els])
for idx, (title, singer) in enumerate(zip(title_els, singer_els)):
    print('{}위 | {} - {}'.format(idx+1,
                                 title.text.strip(),
                                 singer.text))

### 5)네이버 지식IN으로 검색(open API 사용 X)
- 특정 keyword를 특정페이지 수(3)만큼 검색한 결과 데이터프레임으로 출력

In [ ]:
# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword = '챗지피티'
url = 'https://kin.naver.com/search/list.naver?query={}'.format(keyword)
# print(url)
response = get(url)
# response.status_code
soup = BeautifulSoup(response.text, # response.content
                    'html.parser')
soup

In [ ]:
# 방법2
from urllib.request import urlopen
from urllib.parse import quote
# 쳇지피티 -> %EC%B3%87%EC%A7%80%ED%94%BC%ED%8B%B0
keyword = quote(keyword) # url 인코딩된 문자열로 전환
url = 'https://kin.naver.com/search/list.naver?query={}'.format(keyword)
# print(url)
response = urlopen(url)
soup = BeautifulSoup(response, # response.read()
                     'html.parser')

In [ ]:
# 페이징 포함(keyword를 pages수 만큼 검색한 결과를 데이터프레임에 )
import requests
from bs4 import BeautifulSoup
import pandas as pd
keyword = '쳇지피티'
pages = 3
items_list = [] # 크롤링 한 데이터를 담을 list(2차원 리스트)
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    # print(url)
    # 해당 url의 soup객체 -> 글제목, link를 가져와서 items_list에 append
    response = requests.get(url)
    # print(response.status_code) 정상 접근 가능한지 여부
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.select('dt > a')
    # print([item.attrs['href'] for item in items])
    for item in items:
        items_list.append([item.text, item.attrs.get('href')])
#         items_list.append({
#                 'title':item.text,
#                 'link':item.attrs.get('href')})
# print(len(items_list), '개 데이터')
# print(items_list[:2])
df = pd.DataFrame(items_list, columns=['title', 'link'])
df

In [ ]:
# 페이징 포함(keyword를 pages수 만큼 검색한 결과를 데이터프레임에 )
import requests
from bs4 import BeautifulSoup
import pandas as pd
keyword = '쳇지피티'
pages = 3
items_list = [] # 크롤링 한 데이터를 담을 list(2차원 리스트)
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    # print(url)
    # 해당 url의 soup객체 -> 글제목, link를 가져와서 items_list에 append
    response = requests.get(url)
    # print(response.status_code) 정상 접근 가능한지 여부
    soup = BeautifulSoup(response.text, 'html.parser')
    dts = soup.find_all('dt')
    # print([item.attrs['href'] for item in items])
    for dt in dts:
        item = dt.find('a')
        items_list.append([item.text, item.attrs.get('href')])
df = pd.DataFrame(items_list, columns=['title', 'link'])
df

### 6)네이버 지식IN으로 검색(open API 사용 O)
- 특정 keyword를 특정페이지 수(30)만큼 검색한 결과 데이터프레임으로 출력
    - .env에 발급받은 키 저장 -> load (pip install python-dotenv)

In [ ]:
# 환경변수를 가져오기
from dotenv import load_dotenv
import os
load_dotenv() # dotenv_path='.env' 기본값
# print(os.getenv('Client_ID'))
# print(os.getenv('Client_Secret'))

In [ ]:
# 네이버 검색 API 예제 - 지식인 검색
import os
import sys
import urllib.request
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')
encText = urllib.parse.quote("쳇지피티")
url = "https://openapi.naver.com/v1/search/kin.json?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/kin.xml?query=" + encText # XML 결과

headers = {'X-NAVER-Client-ID':client_id,
          'X-NAVER-Client-Secret':client_secret}
request = urllib.request.Request(url, headers=headers)
# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)

response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(type(response_body.decode('utf-8')))
    import json
    data = json.loads(response_body.decode('utf-8')) # json형태의 str을  딕셔너리
    print(type(data))
    print(data['items'])
else:
    print("Error Code:" + rescode)

In [ ]:
# keyword를 display수만큼 지식in검색 결과를 데이터프레임에 (방법1 크롤링)
import requests
import pandas as pd
import json 
from dotenv import load_dotenv
import os
load_dotenv()
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')
keyword = '쳇지피티'
cnt = 30
url = f'https://openapi.naver.com/v1/search/kin.json?query={keyword}&display={cnt}'
headers = {'X-NAVER-Client-ID':client_id,
          'X-NAVER-Client-Secret':client_secret}
response = requests.get(url, headers=headers)
# response.status_code
# items = json.loads(response.text)['items'] : json.loads() -방법1과 방법2
items = response.json()['items'] # 방법1에서만 가능한 함수
items_list = []
for item in items:
    items_list.append([item.get('title').replace('<b>','').replace('</b>',''), # item['title']
                       item.get('link'),
                       item.get('description')
                      ])
df = pd.DataFrame(items_list, columns=['title', 'link', 'description'])
df.head()

In [ ]:
# 방법 2
def get_naver_kin(keyword, cnt):
    import requests
    import pandas as pd
    import json 
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('Client_ID')
    client_secret = os.getenv('Client_Secret')
    keyword = '쳇지피티'
    cnt = 30
    url = f'https://openapi.naver.com/v1/search/kin.json?query={keyword}&display={cnt}'
    headers = {'X-NAVER-Client-ID':client_id,
              'X-NAVER-Client-Secret':client_secret}
    response = requests.get(url, headers=headers)
    # response.status_code
    # items = json.loads(response.text)['items'] : json.loads() -방법1과 방법2
    items = response.json()['items'] # 방법1에서만 가능한 함수
    items_list = []
    for item in items:
        items_list.append([item.get('title').replace('<b>','').replace('</b>',''), # item['title']
                           item.get('link'),
                           item.get('description').replace('<b>','').replace('<b>','')
                          ])
    return pd.DataFrame(items_list, columns=['title', 'link', 'description'])

In [ ]:
get_naver_kin("청바지",20).head()

### quiz) 네이버 open API를 이용해서 청바지 이미지 100건 데이터를 csv파일로 저장
        데이터 : 제목, 이미지링크, 썸네일링크, 높이, 폭

In [ ]:
def get_naver_image(keyword, cnt):
    import requests
    import pandas as pd
    import json
    from dotenv import load_dotenv
    import os

    load_dotenv()
    client_id = os.getenv('Client_ID')
    client_secret = os.getenv('Client_Secret')
    
    keyword = '청바지 이미지'
    cnt = 100

    url = f'https://openapi.naver.com/v1/search/image?query={keyword}&display={cnt}'
    headers = {'X-Naver-Client-Id': client_id,
              'X-Naver-Client-Secret': client_secret}

    response = requests.get(url, headers=headers)
    items = response.json()['items']
    for item in items:
        items_list.append([
            item.get('title').replace('<b>', '').replace('</b>', ''),
            item.get('link'),
            item.get('thumbnail'),
            item.get('sizeheight'),
            item.get('sizewidth')
        ])

    return pd.DataFrame(items_list, columns=['title', 'link', 'thumbnail', 'height', 'width'])

In [ ]:
df = get_naver_image('청바지', 100)
df.head()

In [ ]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
load_dotenv() 
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')
word = "청바지"
url = f'https://openapi.naver.com/v1/search/image?query={word}&display=100'
headers = {'X-Naver-Client-Id': client_id,
           'X-Naver-Client-Secret':client_secret}
response = requests.get(url, headers=headers)
# print(response.text)
items = response.json()['items'] # 방법1에서만 가능한 함수
items_list = []
for idx, item in enumerate(items):
    title = item.get('title')
    link = item.get('link')
    thumbnail = item.get('thumbnail')
    sizeheight = item.get('sizeheight')
    sizewidth = item.get('sizewidth')
    items_list.append({
        'no': idx+1,
        'title': title,
        'link' : link,
        'thumbnail' : thumbnail,
        'sizeheight':sizeheight,
        'sizewidth' :sizewidth
    })
    # link와 thumbnail을 저장
df = pd.DataFrame(items_list)
df.head()

In [ ]:
df.loc[3, 'link']

In [ ]:
df[df['link'].str.find('?')!=-1]

In [ ]:
df[~df['link'].str.endswith('jpg')].loc[5, 'link']

In [ ]:
def save_img(attr, idx, query, url):
    import requests
    '{attr}_{idx}_{query}.{확장자} 이미지 저장'
    file_extension = url.split('.')[-1]
    quote_index = file_extension.find('?')
    
    if quote_index != -1:
        file_extension = file_extension[:quote_index]
#     print(file_extension)
    img = requests.get(url).content
    open(f'ch14_image/{attr}_{idx}_{query}.{file_extension}', 'wb').write(img)

In [ ]:
save_img('메인', 1, '청바지', df.loc[5, 'link'])

In [ ]:
# word image 100개 검색한 데이터는 csv파일로, 이미지는 ch14_image폴더에 저장되는 함수
def get_naver_save_image(word):
    import requests
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv() 
    client_id = os.getenv('Client_ID')
    client_secret = os.getenv('Client_Secret')
    url = f'https://openapi.naver.com/v1/search/image?query={word}&display=100'
    headers = {'X-Naver-Client-Id': client_id,
               'X-Naver-Client-Secret':client_secret}
    response = requests.get(url, headers=headers)

    items = response.json()['items'] # 방법1에서만 가능한 함수
    items_list = []
    for idx, item in enumerate(items):
        title = item.get('title')
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        sizeheight = item.get('sizeheight')
        sizewidth = item.get('sizewidth')
        items_list.append({
            'no': idx+1,
            'title': title,
            'link' : link,
            'thumbnail' : thumbnail,
            'sizeheight':sizeheight,
            'sizewidth' :sizewidth
        })
        # link와 thumbnail을 저장
        save_img('메인', idx+1, word, link)
        save_img('썸네일', idx+1, word, thumbnail)
        # 20%, 40%, 60%, 80% 지점 message 출력
        if idx/20 == round(idx/20):
            print(f'= = = {idx}% 진행 중입니다. = = =')
    df = pd.DataFrame(items_list)
    df.to_csv(f'ch14_image/{word}.csv', encoding='cp949', index=False)
    print('~ ~ ~ 이미지 및 csv 파일 저장 완료 ~ ~ ~')

In [ ]:
get_naver_save_image('청바지')

## 2.3 xml 파일
### 1) 기상예측 RSS 활용
- http://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20250529.xml

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = 'http://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20250529.xml'
response = urlopen(url)
#response.status #response.getcode()
soup = BeautifulSoup(response, "xml") # pip install lxml
locals = soup.find_all('local_ta')
# locals[1]
items_list = []
for local in locals:
    local_ta_name = local.select_one('local_ta_name')
    week1_normalYear = local.select_one('week1_local_ta_normalYear') # 평년기온
    week1_similarRange = local.select_one('week1_local_ta_similarRange') # 기온범위
    week1_min = local.select_one('week1_local_ta_minVal')
    week1_max = local.select_one('week1_local_ta_maxVal')
    week2_normalYear = local.select_one('week2_local_ta_normalYear') # 평년기온
    week2_similarRange = local.select_one('week2_local_ta_similarRange') # 기온범위
    week2_min = local.select_one('week2_local_ta_minVal')
    week2_min = local.select_one('week2_local_ta_maxVal')
    items_list.append({
        '지역' : local_ta_name,
        '1주평년기온' : week1_normalYear,
        '1주 범위' : week1_similarRange,
        '1주 최저' : week1_min,
        '1주 최고' : week1_max,
        '2주평년기온' : week2_normalYear,
        '2주 범위' : week2_similarRange,
        '2주 최저' : week2_min,
        '2주 최고' : week2_min
    })
df = pd.DataFrame(items_list)
df.info()

# 3. 연습문제
- yes24의 베스트셀러 정보를 ch14_yes24.csv(여유가 된다면 ch14_yes24.txt)로 출력
- 작업순서 : 순위(1~48위까지), 책이름, 저장, 출판사, 가격 = > 데이터프레임 => csv => txt

In [ ]:
# import
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# 방법1
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    response = requests.get(url)
    # print(url)
    # print(response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks = [int(ranks_el.text) for ranks_el in ranks_els]
    titles_els=soup.select("div.item_info > div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    writers_els = soup.select("span.authPub.info_auth")
    # writers_els = soup.find_all('span', class_='info_auth')
    writers = [writers_el.text.strip() for writers_el in writers_els]
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, writer, publisher, price in zip(ranks, titles, writers, publishers, prices):
            # print("{},{},{},{},{}".format(rank, title, writer, publisher, price))
            f.write(f'{rank}, {title}, {writer}, {publisher}, {price}\n')
            bestseller_list.append([rank, title, writer, publisher, price])
df = pd.DataFrame(bestseller_list, 
                  columns=['rank', 'title', 'writer', 'publisher', 'price'])
df.to_csv('data/ch14_yes24.csv', index=False)

In [ ]:
# 방법2
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    # print(url)
    response = urlopen(url)
    #print(response.status)
    soup = BeautifulSoup(response, 'html.parser')
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks = [int(ranks_el.text) for ranks_el in ranks_els]
    titles_els=soup.select("div.item_info > div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    writers_els = soup.select("span.authPub.info_auth")
    # writers_els = soup.find_all('span', class_='info_auth')
    writers = [writers_el.text.strip() for writers_el in writers_els]
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, writer, publisher, price in zip(ranks, titles, writers, publishers, prices):
            # print("{},{},{},{},{}".format(rank, title, writer, publisher, price))
            bestseller_list.append({
                'rank':rank, 
                'title':title,
                'writer':writer,
                'publisher':publisher,
                'price':price})
            f.write(f'{rank}, {title}, {writer}, {publisher}, {price}\n')
df = pd.DataFrame(bestseller_list)
df.to_csv('data/ch14_yes24.csv', index=False)

### 2) 버스위치정보 조회(OpenAPI활용)
- data.go.kr
    * 서울특별시_노선정보조회 서비스(busRouteId, stationNm-정류소명, station-정류소 id)
    * 서울특별시_버스위치정보조회 서비스(busRouteId->버스들위치)

In [12]:
# step1 : 버스번호 -> busRouteId
# 노선정보조회 서비스 3번 getBusRouteList서비스

In [11]:
from dotenv import load_dotenv
import os
load_dotenv() # 디폴트값 : dotenv_path='.env'
# print('key = ', os.getenv('key'))
# print('skey = ', os.getenv('skey'))

True

In [25]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlretrieve # url 경로파일을 내 pc에 저장
from urllib.parse import quote
busNum = quote('마포01')
busNum = '162'
key = os.getenv('key')

url1 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
# print(url)
#response = requests.get(url)
#soup = BeautifulSoup(response.text, "xml")
savefilename = 'data/ch14_busInfo.xml'
urlretrieve(url1, savefilename) #url경로 파일을 savefilename으로 저장
xml = open(savefilename, 'r', encoding='utf-8').read()
soup = BeautifulSoup(xml, "xml")

for item in soup.select('itemList'):
    busRouteNm = item.select_one('busRouteNm').text
    if busRouteNm == busNum:
        busRouteId = item.select_one('busRouteId').text
        break
print('busRouteId :', busRouteId)

busRouteId : 100100034


In [26]:
# step2 : busRouteId로 버스 정류장 목록(정류장이름과 정류장 id)
# 노선정보조회 서비스 4번 getStaionsByRouteList

In [38]:
import pandas as pd
url2 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
print(url2)
response = requests.get(url2)
soup = BeautifulSoup(response.text, "xml")
itemList = soup.find_all('itemList')
print(busNum, '번 정류장 갯수 :', len(itemList))
bus_station = []
for itemList in itemList:
    # 정류장이름, 정류장 id, 경도, 위도
    stationNm = itemList.find('stationNm').text
    station = itemList.find('station').text
    gpsX = itemList.find('gpsX').text
    gpsY = itemList.find('gpsY').text
    bus_station.append([stationNm, station, gpsX, gpsY])
df = pd.DataFrame(bus_station, columns=['정류소명', 'id', '경도', '위도'])
df.head()

http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey=kWiLqR5an3qgHfkycjPmjxAie5mGGzW0LuPktrSNA3kqpsmWfsLSW9TxR9k0khrZgJLecVcO88n5LsStz845eg%3D%3D&busRouteId=100100034
162 번 정류장 갯수 : 77


,정류소명,id,경도,위도
0,정릉산장아파트,107000071,127.003343,37.616712
1,정릉4동주민센터.경국사,107000073,127.006345,37.613529
2,북한산보국문역2번출구,107000518,127.0079858233,37.612293899
3,성북청수도서관.정릉4동성당,107000075,127.0084193769,37.6115696748
4,정릉시장입구,107000077,127.0098212542,37.6084653256


In [41]:
# step3 : busRouteId로 버스 위치 정보 조회 2번 getBusPosByRtidList

In [48]:
url3 = f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey={key}&busRouteId={busRouteId}'
print(url3)
response = requests.get(url3)
soup = BeautifulSoup(response.text, "xml")
bus_position = []
itemList = soup.select('itemList')
print('운행중인 버스 수:', len(itemList))
for itemList in soup.select('itemList'):
    # 차량번호, 혼잡도, 경도, 위도, 최종정류장id(lastStnId), 다음정류장 id, 도착소요시간
    plainNo = itemList.select_one('plainNo').text
    # print(plainNo)
    congetion = itemList.select_one('congetion').text
    congetion = '여유' if congetion == '3' \
            else "보통" if congetion=='4' \
            else "혼잡" if congetion=='5' \
            else '매우혼잡'
    # print(congetion)
    gpsX = itemList.select_one('gpsX').text
    gpsY = itemList.select_one('gpsY').text
    lastStnId = itemList.select_one('lastStnId').text
    nextStId = itemList.select_one('nextStId').text
    nextStTm = itemList.select_one('nextStTm').text
    bus_position.append({
        '차량번호' : plainNo,
        '혼잡도' : congetion,
        '경도' : gpsX,
        '위도' : gpsY,
        '최종정류소id' : lastStnId,
        '다음정류소id' : nextStId,
        '도착소요시간' : nextStTm
    })
df_pos = pd.DataFrame(bus_position)
df_pos.head()

http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey=kWiLqR5an3qgHfkycjPmjxAie5mGGzW0LuPktrSNA3kqpsmWfsLSW9TxR9k0khrZgJLecVcO88n5LsStz845eg%3D%3D&busRouteId=100100034
운행중인 버스 수: 21


,차량번호,혼잡도,경도,위도,최종정류소id,다음정류소id,도착소요시간
0,서울70사6556,여유,127.01146,37.605178,107000079,107000168,103
1,서울74사3008,여유,127.016139,37.59374,107000174,101000042,1399
2,서울74사1537,여유,126.990508,37.577498,100000002,101000042,833
3,서울74사1618,여유,126.982914,37.572228,100000106,101000042,324
4,서울74사3372,여유,126.972538,37.557193,101000008,102000018,697


In [53]:
# id가 107000079인 정류소명
df.loc[df['id'] == '107000079', '정류소명'].iloc[0]

'정릉우체국앞'

In [61]:
def station_name(row):
    row['최종정류소명'] = df.loc[df['id'] == row['최종정류소id'], '정류소명'].iloc[0]
    row['다음정류소명'] = df.loc[df['id'] == row['다음정류소id'], '정류소명'].iloc[0]
    return row

In [62]:
station_name(df_pos.loc[0])

차량번호       서울70사6556
혼잡도               여유
경도         127.01146
위도         37.605178
최종정류소id    107000079
다음정류소id    107000168
도착소요시간           103
최종정류소명        정릉우체국앞
다음정류소명      정릉입구.정릉역
Name: 0, dtype: object

In [64]:
df_pos = df_pos.apply(station_name, axis=1)
df_pos.samplemple()

,차량번호,혼잡도,경도,위도,최종정류소id,다음정류소id,도착소요시간,최종정류소명,다음정류소명
1,서울74사3008,여유,127.016139,37.59374,107000174,101000042,1399,성신여대입구역,해운센터.롯데영플라자


In [70]:
# 컬럼명에 id가 포함된 컬럼 제거
drop_true = df_pos.columns.str.contains('id')
drop_column_name = df_pos.columns[drop_true]
df_pos.drop(drop_column_name, axis=1)

,차량번호,혼잡도,경도,위도,도착소요시간,최종정류소명,다음정류소명
0,서울70사6556,여유,127.01146,37.605178,103,정릉우체국앞,정릉입구.정릉역
1,서울74사3008,여유,127.016139,37.59374,1399,성신여대입구역,해운센터.롯데영플라자
2,서울74사1537,여유,126.990508,37.577498,833,창경궁.서울대학교병원,해운센터.롯데영플라자
3,서울74사1618,여유,126.982914,37.572228,324,조계사.종로경찰서,해운센터.롯데영플라자
4,서울74사3372,여유,126.972538,37.557193,697,숭례문,남영역
5,서울70사6557,여유,126.970843,37.541809,837,남영역,시범아파트.대교아파트
6,서울74사3366,여유,126.947522,37.52812,195,원효로3가,시범아파트.대교아파트
7,서울74사1604,여유,126.926857,37.519012,1001,샛강역1번출구.여의도자이,여의도역6번출구
8,서울70사6549,여유,126.924356,37.521786,897,여의도역5번출구,여의도역6번출구
9,서울74사2296,여유,126.92323,37.522601,215,한국경제인협회,여의도역6번출구
